# Getting Started with Arrays in BigQuery
Arrays are an important topic to understand on at least a basic level to work with data in BigQuery. It allows additional information to be stored in lists or key-value pairs within the table. Both can be searched on and extracted to find exactly what you are looking for.

First you need to install and prepare the environment again.

In [ ]:
# These commands were run when the notebook started but are shown in case you choose to work with Notebooks later.
!pip install google.cloud.bigquery
!pip install google.cloud.storage
!pip3 install --upgrade google-cloud-bigquery
!pip install google --user
!pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]' --user

In [ ]:
# Run this command block to install the BigQuery extension.
%load_ext google.cloud.bigquery

# Problem : I want to find RNA-Seq data from estrogen receptor positive breast cancer cell lines.
Doing a search like this will require looking into the attributes stored in the database for each record. If you are an experienced user of SRA you might already know how to do a search like this with Entrez and Run Selector. Here we will find the results in a single SQL search primarily using the attributes column. The methods used to do this can be applied to build very targeted searches of submitter supplied information.

## What Does an Array look like in BigQuery?
Here you will look at a couple of columns from the database to show examples of arrays. You will select the accession column and the columns that are either simple arrays or are an array of data structures. 

In [ ]:
%%bigquery

SELECT acc, datastore_filetype, datastore_provider, datastore_region, attributes
FROM `nih-sra-datastore.sra.metadata`
WHERE organism = 'Homo sapiens'
LIMIT 5

## Viewing what is in the Attributes Column of an SRA Record
To understand what is in the attributes column in the database it may help to view the contents as a table. All of the columns other than **acc** are arrays \[ \] with a comma separated list of items inside. The **attributes** column is an array of structs { } that have a key (k) and a value (v) for each item in the array. You will use the UNNEST function in BigQuery to extract the contents of the attributes column into one item per row. 

## Listing the Contents of an Array
If you unnest the attributes column, you get the list of the key-value pairs. The contents of this column are some standard items like "bytes" and "bases" as well as items that were provided by the submitter as additional information. This means that runs in the database will have some attributes in common but also some attributes not on other runs. There can also be keys that are repeated so one key might have multiple values in a single run. 

### Naming Unnested Data
When you unnest the data you can give the result of the unnest function a name (unnested_attributes in the below query) and then use that name in the select statement. Like you did before when naming tables, it can be used to make queries a little easier to type and read.

In [ ]:
%%bigquery

SELECT unnested_attributes
FROM `nih-sra-datastore.sra.metadata`,
    UNNEST(attributes) as unnested_attributes 
WHERE acc = 'SRR2973262'

## Listing the Keys and Values in the Attributes as a Table
You can further break a structured list like this {'k': 'sex_calc', 'v': 'female'} apart into (k) and (v) columns by using unnested_attributes.k and unnested_attributes.v in the query. At this point you will be seeing all the keys and values in a table format for a single accession in the metadata table.

In [ ]:
%%bigquery

SELECT unnested_attributes.k, unnested_attributes.v
FROM `nih-sra-datastore.sra.metadata`,
    UNNEST(attributes) as unnested_attributes 
WHERE acc = 'SRR2973262'

## Finding Runs with a Certain Attribute
Listing out all the attribute keys and values can be quite useful to help build searches for very specific things. Looking at the results for the above query, one of the attributes is tissue_sam : colon. If you want to find all runs where that exists you can use the following query.

In [ ]:
%%bigquery

SELECT acc
FROM `nih-sra-datastore.sra.metadata`   
WHERE ('tissue_sam', 'colon') in UNNEST(attributes)

## Search for Breast Cancer Cell Line Records
Now you can use the UNNEST function to search for the data in the initial problem. The LIKE function with the wildcard % in will let you search for records that contain certain words or phrases with anything around them. The query below will search for all records that mention "breast" and "cancer" and "cell" and "line" in at least one attribute.

In [ ]:
%%bigquery

SELECT acc, assay_type, consent, librarysource, libraryselection, bioproject,
FROM `nih-sra-datastore.sra.metadata` meta,
    UNNEST(attributes) as extracted
WHERE extracted.v LIKE '%breast%cancer%cell%line%'

### Subselect
A subselect is a select statement embedded within another select statement. In this case you will use a subselect to get the value of attributes using the attribute key name. This example uses 'source_name_sam' to see values that are on *some* records and 'bases' which will be on all records.

You can also find the key for the attribute that contained the breast cancer cell line entry by using a subselect. Because there might be more than one attribute key that contains that phrase in a single record, you can use the limit function to just find the first hit. You could do something like build an array from all the key hits but that is more involved and not necessary in this example.

In [ ]:
%%bigquery

SELECT acc, assay_type, consent, librarysource, libraryselection, bioproject,
    (select v from unnest(attributes) where k = 'source_name_sam')as source_name,
    (select v from unnest(attributes) where k = 'bases')as bases,
    (select k from unnest(attributes) where v like '%breast%cancer%cell%line%' limit 1) as breast_cancer_attribute,
FROM `nih-sra-datastore.sra.metadata` meta,
    UNNEST(attributes) as extracted
WHERE extracted.v LIKE '%breast%cancer%cell%line%'

## Limiting the Search Further
Now you will add the search about the Estrogen Receptor status of the cells as well as limit it to paired RNA-Seq datasets using cDNA selection that are in the public domain. And again you will query the keys to see what attribute contained the information about the cells being ER-positive.
# paired RNA-Seq datasets using cDNA selection that are in the public domain and

In [ ]:
%%bigquery

SELECT acc, assay_type, consent, librarysource, libraryselection, bioproject,
    (select v from unnest(attributes) where k = 'source_name_sam')as source_name,
    (select v from unnest(attributes) where k = 'bases')as bases,
    (select k from unnest(attributes) where v like '%ER-positive%' limit 1) as ER_column
FROM `nih-sra-datastore.sra.metadata` meta,
    UNNEST(attributes) as extracted
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND libraryselection = 'cDNA'
    AND librarylayout = 'PAIRED'
    AND extracted.v LIKE '%breast%cancer%cell%line%'
    AND extracted.v LIKE '%ER-positive%'

## Storing Accessions or Metadata Tables in a Variable
In the last couple of examples we will look at storing query output in a file within Jupyter. You can store the results of the query in a variable using a function in the bigquery module. You do this by providing the name of the variable you will use ("metadata" in this example) to store the output in. This is somewhat specific to the bigquery python module you're using here but the there may be some overlap with other python methods you may be using for work.

First we will store the metadata table from the previouls in one variable then we will store just the accessions in a different variable.

In [ ]:
%%bigquery metadata

SELECT acc, assay_type, consent, librarysource, libraryselection, bioproject,
    (select v from unnest(attributes) where k = 'source_name_sam')as source_name,
    (select v from unnest(attributes) where k = 'bases')as bases,
    (select k from unnest(attributes) where v like '%ER-positive%' limit 1) as ER_column
FROM `nih-sra-datastore.sra.metadata` meta,
    UNNEST(attributes) as extracted
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND libraryselection = 'cDNA'
    AND librarylayout = 'PAIRED'
    AND extracted.v LIKE '%breast%cancer%cell%line%'
    AND extracted.v LIKE '%ER-positive%'

In [ ]:
%%bigquery accessions

SELECT acc
FROM `nih-sra-datastore.sra.metadata` meta,
    UNNEST(attributes) as extracted
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND libraryselection = 'cDNA'
    AND librarylayout = 'PAIRED'
    AND extracted.v LIKE '%breast%cancer%cell%line%'
    AND extracted.v LIKE '%ER-positive%'


## Viewing the Contents of a Variable
You can output the contents of a variable by running a cell with just the variable name in it.

In [ ]:
metadata

In [ ]:
accessions

## Storing the Variable Contents as a File
The results variable contains a dataframe with an index (0-5) and a header (acc) that you don't want in the accession list. You will remove those using the to_string function using the options of index=False and header=False. This will save a file with just a list of accessions. Alternatively if you wanted to save the results of a metadata query that included the header information, you can omit the header=False option.

In [ ]:
file = open("accessions.txt", "w")
file.write(accessions.to_string(index=False, header=False))
file.close()

file = open("metadata.txt", "w")
file.write(metadata.to_string())
file.close()